In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import random_uuid, invoke_graph
from graph import DataExtractor

In [2]:
# .env 파일 로드
load_dotenv(dotenv_path=".env")

# API 키 가져오기
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")

# LangSmith 추적 기능을 활성화합니다. (선택적)
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [3]:
voltai_graph = DataExtractor(
    folder_path="./data/input_data/", 
    file_number=56
).graph

In [4]:
# config 설정(재귀 최대 횟수, thread_id)
config = RunnableConfig(recursion_limit=20, configurable={"thread_id": random_uuid()})

In [5]:
# 4개의 질문 입력
questions = [
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "CAM (Cathode Active Material)": {
                "Stoichiometry information": "NCM-622",
                "Commercial NCM": "No",
                "Lithium source": "LiOH",
                "Synthesis method": "co-precipitation",
                "Crystallization method": "Hydrothermal",
                "Crystallization temperature": "100°C",
                "Crystallization time": "12 hr",
                "Doping": "Zr4+ doping",
                "Coating": "ZrO2 coating",
                "Additional treatment": "None"
                }
        }
    ]
    """,    
    
    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {
            "Electrode (only for coin-cell (half-cell))": {
                "Active material : Conductive additive : Binder ratio": "90 : 5 : 5",
                "Electrolyte": "LiPF6 (EC, EMC, DEC mixture in a 1:1:1 volume ratio)",
                "Additive": "FEC 10% addition",
                "Electrode thickness": "100 µm",
                "Only Cathode Electrode diameter": "14π",
                "Loading density (mass loading of NCM)": "0.005 g/cm^2",
                "Additional treatment for electrode": "None"
                },
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {        
            "Morphological results": {
                "Explanation of SEM results": "Fig. 2a, b; the NCM-622 seems to have more or less a spherical morphology with a diameter of 3–5 µm, composed of densely packed primary particles",
                "Explanation of TEM results": "None"
                },    
        }
    ]
    """,

    """Below are instructions for filling out items by referring to the examples.
    The values shown to the right of the colon (“:”) are examples;
    please delete them after reviewing and rewrite them with the values found in the PDF.
    If any item is not mentioned in the PDF, do not remove it—write “None.”
    [
        {            
            "Cathode Performance": {
                "Capacity at all C-rate, mAh/g (with electrode state)": [{
                "214.5 mAh/g": "@0.1C, ZrO2-coated",
                "200.8 mAh/g": "@0.5C, ZrO2-coated"
                }],
                "Voltage range": "2.8–4.3 V",
                "Temperature": "Room temperature and 55°C"
                }
        }
    ]
    """
]


In [7]:
# 4개의 질문에 대해 그래프 실행 및 출력
outputs = []
for i, question in enumerate(questions):
    # inputs = GraphState(question=question)
    print(f"#####   {i+1}번째 질문   #####")
    # print(question)
    result = voltai_graph.invoke(
        input={"question":question},
        config=config,
    )
    outputs.append(result)

#####   1번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   2번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   3번째 질문   #####
==== [RELEVANCE CHECK] ====
yes
#####   4번째 질문   #####
==== [RELEVANCE CHECK] ====
yes


In [7]:
outputs

[{'question': 'Below are instructions for filling out items by referring to the examples.\n    The values shown to the right of the colon (“:”) are examples;\n    please delete them after reviewing and rewrite them with the values found in the PDF.\n    If any item is not mentioned in the PDF, do not remove it—write “None.”\n    [\n        {\n            "CAM (Cathode Active Material)": {\n                "Stoichiometry information": "NCM-622",\n                "Commercial NCM": "No",\n                "Lithium source": "LiOH",\n                "Synthesis method": "co-precipitation",\n                "Crystallization method": "Hydrothermal",\n                "Crystallization temperature": "100°C",\n                "Crystallization time": "12 hr",\n                "Doping": "Zr4+ doping",\n                "Coating": "ZrO2 coating",\n                "Additional treatment": "None"\n                }\n        }\n    ]\n    ',
  'context': 'prominently reduced. From Figure 3C, the O 1 s spec

In [8]:
import pprint
outputs_dict = outputs[0]["answer"][0]["CAM (Cathode Active Material)"] | outputs[1]["answer"][0]["Electrode (only for coin-cell (half-cell))"] | outputs[2]["answer"][0]["Morphological results"] | outputs[3]["answer"][0]["Cathode Performance"]
pprint.pprint(outputs_dict)

{'Active material : Conductive additive : Binder ratio': '96 : 2 : 2',
 'Additional treatment': 'None',
 'Additional treatment for electrode': 'None',
 'Additive': 'None',
 'Capacity at all C-rate, mAh/g (with electrode state)': [{'180.0 mAh/g': '@5.0C, '
                                                                          'M-NCM',
                                                           '203.5 mAh/g': '@0.1C, '
                                                                          'M-NCM'}],
 'Coating': 'Li3PO4 coating',
 'Commercial NCM': 'Yes',
 'Crystallization method': 'Calcination and sintering',
 'Crystallization temperature': '750°C',
 'Crystallization time': '15 hr',
 'Doping': 'Mg2+ doping',
 'Electrode thickness': 'None',
 'Electrolyte': '1 M LiPF6 in DMC, EC, and EMC (1:1:1 vol%)',
 'Explanation of SEM results': 'Fig. 9A; the cycled electrode of the pristine '
                               'NCM is damaged by intergranular cracking along '
                        

In [9]:
seperated_outputs = outputs[0]["answer"][0] | outputs[1]["answer"][0] | outputs[2]["answer"][0] | outputs[3]["answer"][0]
pprint.pprint(seperated_outputs)

{'CAM (Cathode Active Material)': {'Additional treatment': 'None',
                                   'Coating': 'Li3PO4 coating',
                                   'Commercial NCM': 'Yes',
                                   'Crystallization method': 'Calcination and '
                                                             'sintering',
                                   'Crystallization temperature': '750°C',
                                   'Crystallization time': '15 hr',
                                   'Doping': 'Mg2+ doping',
                                   'Lithium source': 'LiOH/H2O',
                                   'Stoichiometry information': 'LiNi0.8Co0.1Mn0.1O2',
                                   'Synthesis method': 'Co-precipitation'},
 'Cathode Performance': {'Capacity at all C-rate, mAh/g (with electrode state)': [{'180.0 mAh/g': '@5.0C, '
                                                                                                  'M-NCM',
         

In [10]:
pd.DataFrame(outputs_dict)

,Stoichiometry information,Commercial NCM,Lithium source,Synthesis method,Crystallization method,Crystallization temperature,Crystallization time,Doping,Coating,Additional treatment,...,Additive,Electrode thickness,Only Cathode Electrode diameter,Loading density (mass loading of NCM),Additional treatment for electrode,Explanation of SEM results,Explanation of TEM results,"Capacity at all C-rate, mAh/g (with electrode state)",Voltage range,Temperature
0,LiNi0.8Co0.1Mn0.1O2,Yes,LiOH/H2O,Co-precipitation,Calcination and sintering,750°C,15 hr,Mg2+ doping,Li3PO4 coating,None,...,None,None,None,None,None,Fig. 9A; the cycled electrode of the pristine ...,None,"{'203.5 mAh/g': '@0.1C, M-NCM', '180.0 mAh/g':...",3.0–4.3 V,None
